In [5]:
# In case chromedriver version is not compatitable with browser one
!pip install --upgrade --force-reinstall chromedriver-binary-auto

  Using cached chromedriver_binary_auto-0.2.0-py3-none-any.whl
  Attempting uninstall: chromedriver-binary-auto
    Found existing installation: chromedriver-binary-auto 0.2.0
    Uninstalling chromedriver-binary-auto-0.2.0:
      Successfully uninstalled chromedriver-binary-auto-0.2.0


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Jul\\anaconda3\\Lib\\site-packages\\~hromedriver_binary\\chromedriver.exe'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\users\jul\anaconda3\python.exe -m pip install --upgrade pip' command.


In [6]:
from selenium.webdriver import Chrome
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import chromedriver_binary

In [7]:
driver = Chrome()
driver.get('https://twitter.com/i/flow/login')

In [8]:
username = driver.find_element(By.XPATH,'//input[@name = "text"]')
username.send_keys('nad.yul2002@gmail.com')

In [9]:
username.send_keys(Keys.RETURN)

In [10]:
my_password = getpass()

········


In [11]:
password = driver.find_element(By.XPATH, '//input[@name = "password"]')
password.send_keys(my_password)

In [12]:
password.send_keys(Keys.RETURN)

In [73]:
newspaper_name = ['Bloomberg', 'The Washington Post', 'The New York Times', 'The Wall Street Journal']
file_name = ['Bloomberg', 'WashingtonPost', 'NYT', 'WSJ']
curr_index = 0

In [74]:
search_input = driver.find_element(By.XPATH, '//input[@aria-label="Search query"]')
search_input.send_keys(newspaper_name[curr_index])
search_input.send_keys(Keys.RETURN)

In [78]:
driver.find_element(By.LINK_TEXT, 'People').click()
paper_name_input = driver.find_element(By.XPATH, "//span[text() = '" + newspaper_name[curr_index] + "']").click()

In [79]:
def get_tweet_data(card):
    """Extract data from tweet"""
    username = card.find_element(By.XPATH, './/span').text
    handle = card.find_element(By.XPATH, './/span[contains(text(), "@")]').text
    try:
        postdate = card.find_element(By.XPATH, './/time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element(By.XPATH, './/div[1]/div[1]/div[1]/div[2]/div[2]/div[2]').text
    responding = card.find_element(By.XPATH, './/div[1]/div[1]/div[2]').text
    text = comment+responding
    reply_cnt = card.find_element(By.XPATH, './/div[@data-testid="reply"]').text
    retweet_cnt = card.find_element(By.XPATH, './/div[@data-testid="retweet"]').text
    likes_cnt = card.find_element(By.XPATH, './/div[@data-testid="like"]').text
    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, likes_cnt)
    return tweet

In [80]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

In [81]:
#get all tweets on the page

data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements(By.XPATH, './/article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
    
    scroll_attempt = 0
    while True:            
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1

            if scroll_attempt >=3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = curr_position
            break

In [82]:
data

[('Bloomberg',
  '@business',
  '2022-07-31T14:53:04.000Z',
  'Nigerians accumulating cryptocurrencies to shield assets against a weakening naira have contributed to a slump in the fiat currency to a record low\nbloomberg.com\nCrypto Trades Drive Naira to New Low in Unauthorized Market\nNigerians accumulating cryptocurrencies to shield assets against a weakening naira have contributed to a slump in the fiat currency to a record low in the unauthorized market.\n12\n40\n68Bloomberg\n@business\n·\n2h\nNigerians accumulating cryptocurrencies to shield assets against a weakening naira have contributed to a slump in the fiat currency to a record low\nbloomberg.com\nCrypto Trades Drive Naira to New Low in Unauthorized Market\nNigerians accumulating cryptocurrencies to shield assets against a weakening naira have contributed to a slump in the fiat currency to a record low in the unauthorized market.\n12\n40\n68',
  '12',
  '40',
  '68'),
 ('Bloomberg',
  '@business',
  '2022-07-31T14:48:03.000

In [83]:
#Update the file location, rn it saves to the Jupyter files folder
with open(f'{file_name[curr_index]}_tweets.csv', 'w', newline = '', encoding = 'utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Comments', 'Likes', 'Retweets', 'Text']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)